# Understanding the Temporal Implications of One Timestamp

To understand what one timestamp might represent in our study, we refer again to the dataset available at https://www.kaggle.com/datasets/jackogozaly/steam-player-data.

We use the metric 'average players' as a proxy for addiciton to individual games. For each distinct game, we identified the time at which it reached its peak in average player count, herein referred to as 'peak_popularity_time'. Subsequently, we selected all timestamps post 'peak_popularity_time' and determined the lowest timestamp for each game. This allowed us to calculate the time difference from 'peak_popularity_time' to this lowest timestamp ('time_from_peak_to_lowest'). Please note that this difference can be approximated as "Diminishing Phase" in our model setup, i.e T1->T2. For ease of comparison and consistency, we converted this time difference into months, assuming an average month length of 30.44 days.

This step was repeated for each unique game in the dataset, and the average 'time_from_peak_to_lowest' across all games was calculated. Given that our 'Diminishing Phase' incorporates 10 timestamps, we further divided this average by 10 to correspond an approximate value for each timestamp.

**Conclusion:** The final calculation indicates that, on average, each timestamp in our Agent-Based Model (ABM) is equivalent to approximately 1.13 months. Thus, we can infer that each step in our model corresponds to just over a month of real-world time, providing a tangible interpretation of the model's temporal dimension.

In [1]:
import pandas as pd

# We load the data and select the relevant columns
data = pd.read_csv('/content/Valve_Player_Data.csv')
data = data[['Game_Name', 'Month_Year', 'Avg_players']]

# Converting 'Month_Year' to datetime, if it's not already
data['Month_Year'] = pd.to_datetime(data['Month_Year'])

results = []

# Loop through each unique game
for game in data['Game_Name'].unique():
    # Select data for this game
    game_data = data[data['Game_Name'] == game]

    # Finding the time of peak popularity (max 'Avg_players')
    peak_popularity_time = game_data[game_data['Avg_players'] == game_data['Avg_players'].max()]['Month_Year'].iloc[0]

    # Subset the dataframe to only consider dates after peak popularity
    post_peak_data = game_data[game_data['Month_Year'] > peak_popularity_time]

    # If there are no dates after the peak popularity time, we skip this game
    if post_peak_data.empty:
        continue

    # Finding the time of lowest popularity after the peak ('Avg_players' min after peak)
    lowest_popularity_time = post_peak_data[post_peak_data['Avg_players'] == post_peak_data['Avg_players'].min()]['Month_Year'].iloc[0]

    # Calculating the length of time from peak to lowest popularity
    time_from_peak_to_lowest = (lowest_popularity_time - peak_popularity_time).days

    # Since our data is monthly, we convert days to months
    time_from_peak_to_lowest_in_months = time_from_peak_to_lowest / 30.44  # using the average number of days in a month

    # Append results for this game to the list
    results.append({
        'game_name': game,
        'time_from_peak_to_lowest_in_months': time_from_peak_to_lowest_in_months,
    })

# Converting results to a DataFrame
results_df = pd.DataFrame(results)

print(results_df)

# We calculate the average time from peak to lowest across all games
average_time = results_df['time_from_peak_to_lowest_in_months'].mean()

# We then define one time stamp as one tenth of this average time
one_time_stamp = average_time / 10

print(f"One time stamp corresponds to approximately {one_time_stamp:.2f} months.")


                           game_name  time_from_peak_to_lowest_in_months
0   Counter Strike: Global Offensive                           14.980289
1                             Dota 2                           46.977661
2                PUBG: Battlegrounds                           32.982917
3                       Apex Legends                            1.018397
4                    Team Fortress 2                            2.003942
..                               ...                                 ...
85                     Battlefield 1                            1.018397
86                      Dying Light                             3.022339
87                    Slay the Spire                           20.959264
88                        The Sims 4                            2.036794
89          Divinity: Original Sin 2                            9.986859

[90 rows x 2 columns]
One time stamp corresponds to approximately 1.13 months.
